# Assignment 2 & 3

В этих домашних заданиях вам предстоит решить две задачи:<br>
1. Исследовать, как графовый контекст влияет на качество предсказания модели классификации <b>[15 pts]</b>
2. Определить, какие переходы оказывает наибольшее влияние на качество модели <b>[15 pts]</b>

Решение заданий - jupyter notebook


### Assignment 2 - Графовый контекст

В лекции вам рассказали о некоторых подходах к веторизации цепочек процессов. <br>
"Цепочка процесса" - это направленный граф, описывающий конкретную реализацию процесса (конкретный case id из всей выборки). Вершины и ребра графа определяются с помощью майнеров (например, мы реализовывали Альфа-алгоритм). Совокупность вершин и ребер графа, сформированных майнером, называется <b>контекст</b>. <br>

Контекст будет отличаться в зависимости от майнера, который вы применили. Например, альфа-алгоритм может удалить часть вершин ввиду особенностей работы алгоритма. Контекст влияет на то, какие "соседи" будут у каждой из вершин графа, какие метрики будут характеризовать ребра и вершины. И, разумеется, повлияет на то, какие будут эмбеддинги у каждой "цепочки процесса". <br>



В этом задании необходимо реализовать алгоритм, который получает на вход: <br>
1. `pandas.DataFrame()` со столбцами `case_id, stage, stage_datetime` 
2. `pandas.DataFrame()` со столбцами `case_id, target` (задача бинарной классификации)
3. `string` с указанием `algo_name` 

И возвращает значение метрики `roc_auc` и `pandas.DataFrame()` со столбцами `case_id, target, predicted, algo_name`

#### Рассмотрим пример для векторизации с помощью word2vec

##### Считаем данные

In [ ]:
import pandas as pd
import numpy as np
import pprint

In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sberpm

     |████████████████████████████████| 3.6 MB 8.4 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 786 kB 36.6 MB/s 
     |████████████████████████████████| 10.3 MB 23.6 MB/s 
     |████████████████████████████████| 583 kB 44.7 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 381 kB 42.5 MB/s 
     |████████████████████████████████| 8.2 MB 26.3 MB/s 
  Created wheel for sberpm: filename=sberpm-1.3.0-py3-none-any.whl size=134397 sha256=4bd69b1815c812e9ba5671ca97bd7c95f7ccf5a207dc46fb6d97439ee0613a96
  Stored in directory: /root/.cache/pip/wheels/e7/c6/06/038ee19b16fac1277e41166adf4b682c2b29cc82bfff9a5e94
Successfully built sberpm
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolki

In [ ]:
df = pd.read_csv("assignment_2_sample.csv")
df['stage_datetime'] = pd.to_datetime(df.stage_datetime, format='%Y-%m-%d %H:%M:%S')

In [ ]:
df

,case_id,stage,stage_datetime
0,59f8f1b02a074048ae66485008a516ea,Stage_29,2020-07-28 05:04:15
1,59f8f1b02a074048ae66485008a516ea,Stage_148,2020-08-02 01:52:53
2,59f8f1b02a074048ae66485008a516ea,Stage_1,2020-08-02 03:51:29
3,59f8f1b02a074048ae66485008a516ea,Stage_40,2020-08-02 05:40:56
4,59f8f1b02a074048ae66485008a516ea,Stage_60,2020-08-02 05:53:43
...,...,...,...
894673,773d5dbcd29a452792e65994cb2e931e,Stage_10,2021-07-23 14:32:05
894674,773d5dbcd29a452792e65994cb2e931e,Stage_3,2021-07-23 14:37:40
894675,773d5dbcd29a452792e65994cb2e931e,Stage_0,2021-07-23 14:54:37
894676,773d5dbcd29a452792e65994cb2e931e,Stage_34,2021-07-24 08:26:17


In [ ]:
targets = pd.read_csv("assignment_2_targets.csv")

In [ ]:
targets

,case_id,target
0,604a550439d644718ea6e1693fbf03dc,0.0
1,6dee55b3d7284d18bed3094ee3103812,1.0
2,5f3b2dbc151f4067bf9d898a375e0593,0.0
3,5c4fbe21d35943818d110e7aaec739c7,0.0
4,6b92c6ad123648a4aa8058a348fe3c56,1.0
...,...,...
995,62fa7ce4df934039987bc51d6673ad2e,0.0
996,69e225f9821a46e1a6dda900132f1330,1.0
997,76da2b2f65cf4162bcdb264c0322288e,1.0
998,672861216fdf47a582ffa519db221810,1.0


In [ ]:
# пример одной цепочки процесса и соотв. ей класса

sample = df[df.case_id == df.case_id[0]]
sample_target = targets[targets.case_id == df.case_id[0]]

print(sample, '\n\n')
print(sample_target, '\n\n')

                              case_id      stage      stage_datetime
0    59f8f1b02a074048ae66485008a516ea   Stage_29 2020-07-28 05:04:15
1    59f8f1b02a074048ae66485008a516ea  Stage_148 2020-08-02 01:52:53
2    59f8f1b02a074048ae66485008a516ea    Stage_1 2020-08-02 03:51:29
3    59f8f1b02a074048ae66485008a516ea   Stage_40 2020-08-02 05:40:56
4    59f8f1b02a074048ae66485008a516ea   Stage_60 2020-08-02 05:53:43
..                                ...        ...                 ...
914  59f8f1b02a074048ae66485008a516ea   Stage_65 2021-07-29 10:08:32
915  59f8f1b02a074048ae66485008a516ea   Stage_23 2021-07-29 23:29:55
916  59f8f1b02a074048ae66485008a516ea    Stage_0 2021-07-31 03:25:42
917  59f8f1b02a074048ae66485008a516ea    Stage_7 2021-07-31 05:26:32
918  59f8f1b02a074048ae66485008a516ea    Stage_0 2021-08-04 02:53:54

[919 rows x 3 columns] 


                              case_id  target
803  59f8f1b02a074048ae66485008a516ea     1.0 




#####  Пример: построение эмбеддингов с помощью word2vec и контекста Direct Follower Graph

! Note: Direct follower graph состоит из всех этапов и прямых переходов процесса

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
# Отсортируем этапы процесса по времени для каждого id:
df = df.sort_values(['case_id', 'stage_datetime'])

In [ ]:
# Воспользуемся библиотекой sberPM для построения векторного представления

from sberpm.ml.vectorizer import ProcessWord2Vec, ProcessCountVectorizer
from sberpm import DataHolder

class SimpleDFG:
    def __init__(self, train_df, target_df, algo_name):
        self.train_df = train_df
        self.targets = target_df
        self.algo_name = algo_name
        
    def make_splits(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.embeddings, self.targets.target, test_size=0.3, random_state=0)
        
    def get_embeddings(self):
        data_holder = DataHolder(data=self.train_df, 
                                             id_column='case_id', 
                                             activity_column='stage', 
                                             start_timestamp_column='stage_datetime', 
                                             time_format='%Y-%m-%d %H:%M:%S')
        vectorizer = ProcessWord2Vec(8)
        vectorizer.fit(data_holder = data_holder)
        self.embeddings = vectorizer.transform(data_holder)
        
    def train_estimator(self):        
        self.cls = xgb.XGBClassifier(n_estimators=300, max_depth=5)
        self.cls.fit(self.X_train, self.y_train)
        
    def run_pipeline(self):
        self.get_embeddings()
        print('Embeddings ready \n')
        self.make_splits()
        print('Training started... \n')
        self.train_estimator()
        print(f'ROC-AUC @ validation for {self.algo_name} algorithm is {roc_auc_score(self.y_test, self.cls.predict(self.X_test))}')

In [ ]:
simple_dfg_predictor = SimpleDFG(df, targets, 'w2v')

In [ ]:
# Может занять много времени!
simple_dfg_predictor.run_pipeline()

Embeddings ready 

Training started... 

ROC-AUC @ validation for w2v algorithm is 0.5043633922577758


Как видим, итоговое значение по метрике на валидации получилось ~0.51<br>
w2v, как известно, не учитывает последовательность шагов (не важно направление ребер в графе) - что будет с метрикой, если такое направление учесть?

#####  Задание: построение эмбеддингов с помощью HOPE и контекста Heuristics Miner


В этом задании необходимо реализовать алгоритм, который получает на вход: <br>
1. `pandas.DataFrame()` со столбцами `case_id, stage, stage_datetime` 
2. `pandas.DataFrame()` со столбцами `case_id, target` (задача бинарной классификации)
3. `string` с указанием `algo_name` 

И возвращает значение метрики `roc_auc` и `pandas.DataFrame()` со столбцами `case_id, target, predicted, algo_name`

ВАЖНО: последовательности этапов ABC | ACB должны интерпретироваться как разные последовательности и итоговый эмбеддинг *может* отличаться.<br>

Для реализации можно использовать HOPE и word2mat, о которых говорили в лекции. Или использовать LSTM :)

In [ ]:
# большинство майнеров уже реализовано в sberpm
from sberpm.miners import SimpleMiner, AlphaMiner, AlphaPlusMiner, HeuMiner
from sberpm.ml.vectorizer import HopeVectorizer, katz_index, adamic_adar

In [ ]:
from sberpm.ml.vectorizer._graph_embedding_utils.proximity_measures import rooted_pr
class ContextEmbedder:
    def __init__(self, train_df, target_df, algo_name):
        self.train_df = train_df
        self.targets = target_df
        self.algo_name = algo_name
        
    def make_splits(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.embeddings, self.targets.target, test_size=0.3, random_state=0)
        
    def get_embeddings(self):
        data_holder = DataHolder(data=self.train_df, 
                                             id_column='case_id', 
                                             activity_column='stage', 
                                             start_timestamp_column='stage_datetime', 
                                             time_format='%Y-%m-%d %H:%M:%S')

        vectorizer = HopeVectorizer()
        trace_vectors = vectorizer.transform(data_holder, HeuMiner, 8, katz_index, 3)[0]
        self.embeddings =  np.concatenate([el[0] for el in trace_vectors])

    def train_estimator(self):        
        self.cls = xgb.XGBClassifier(n_estimators=300, max_depth=5)
        self.cls.fit(self.X_train, self.y_train)
        
    def run_pipeline(self):
        self.get_embeddings()
        print('Embeddings ready \n')
        self.make_splits()
        print('Training started... \n')
        self.train_estimator()
        print(f'ROC-AUC @ validation for {self.algo_name} algorithm is {roc_auc_score(self.y_test, self.cls.predict(self.X_test))}')

In [ ]:
context_embedder_predictor = ContextEmbedder(df, targets, 'w2m')

In [ ]:
context_embedder_predictor.run_pipeline()

Embeddings ready 

Training started... 

ROC-AUC @ validation for w2m algorithm is 0.5325576191541731


In [ ]:
df

,case_id,stage,stage_datetime
0,59f8f1b02a074048ae66485008a516ea,Stage_29,2020-07-28 05:04:15
1,59f8f1b02a074048ae66485008a516ea,Stage_148,2020-08-02 01:52:53
2,59f8f1b02a074048ae66485008a516ea,Stage_1,2020-08-02 03:51:29
3,59f8f1b02a074048ae66485008a516ea,Stage_40,2020-08-02 05:40:56
4,59f8f1b02a074048ae66485008a516ea,Stage_60,2020-08-02 05:53:43
...,...,...,...
894673,773d5dbcd29a452792e65994cb2e931e,Stage_10,2021-07-23 14:32:05
894674,773d5dbcd29a452792e65994cb2e931e,Stage_3,2021-07-23 14:37:40
894675,773d5dbcd29a452792e65994cb2e931e,Stage_0,2021-07-23 14:54:37
894676,773d5dbcd29a452792e65994cb2e931e,Stage_34,2021-07-24 08:26:17
